# Introduction
LangChain is a framework for developing applications powered by language models. It enables applications that are:

    Data-aware: connect a language model to other sources of data
    Agentic: allow a language model to interact with its environment

The main value props of LangChain are:

    1. Components: abstractions for working with language models, along with a collection of implementations for each   abstraction. Components are modular and easy-to-use, whether you are using the rest of the LangChain framework or not

    2. Off-the-shelf chains: a structured assembly of components for accomplishing specific higher-level tasks
    Off-the-shelf chains make it easy to get started. For more complex applications and nuanced use-cases, components make it   easy   to customize existing chains or build new ones.

# Get started

In [1]:
#install this library
#!pip install langchain
#!pip install openai

# Environment setup

In [6]:
import openai
import os
openai.api_key = 'Enter-API-KEY'

In [7]:
from langchain.llms import OpenAI

llm = OpenAI(openai_api_key="Enter-API-KEY")

# Building an application
Now we can start building our language model application. LangChain provides many modules that can be used to build language model applications. Modules can be used as stand-alones in simple applications and they can be combined for more complex use cases.

# LLMs
### Get predictions from a language model
The basic building block of LangChain is the LLM, which takes in text and generates more text.

As an example, suppose we're building an application that generates a company name based on a company description. In order to do this, we need to initialize an OpenAI model wrapper. In this case, since we want the outputs to be MORE random, we'll initialize our model with a HIGH temperature.

In [9]:
from langchain.llms import OpenAI

llm = OpenAI(openai_api_key="Enter-API-KEY",temperature=0.9)

And now we can pass in text and get predictions!

In [10]:
llm.predict("What would be a good company name for a company that makes colorful socks?")

'\n\nColorful Cozies.'

# Chat models

Chat models are a variation on language models. While chat models use language models under the hood, the interface they expose is a bit different: rather than expose a "text in, text out" API, they expose an interface where "chat messages" are the inputs and outputs.

You can get chat completions by passing one or more messages to the chat model. The response will be a message. The types of messages currently supported in LangChain are AIMessage, HumanMessage, SystemMessage, and ChatMessage -- ChatMessage takes in an arbitrary role parameter. Most of the time, you'll just be dealing with HumanMessage, AIMessage, and SystemMessage.

In [11]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

In [13]:
chat = ChatOpenAI(openai_api_key="Enter-API-KEY",temperature=0)
chat.predict_messages([HumanMessage(content="Translate this sentence from English to French. I love programming.")])

AIMessage(lc_kwargs={'content': "J'aime programmer."}, content="J'aime programmer.", additional_kwargs={}, example=False)

It is useful to understand how chat models are different from a normal LLM, but it can often be handy to just be able to treat them the same. LangChain makes that easy by also exposing an interface through which you can interact with a chat model as you would a normal LLM. You can access this through the predict interface.

In [14]:
chat.predict("Translate this sentence from English to French. I love programming.")
# >> J'aime programmer

"J'aime programmer."

# Prompt templates
Most LLM applications do not pass user input directly into to an LLM. Usually they will add the user input to a larger piece of text, called a prompt template, that provides additional context on the specific task at hand.

In the previous example, the text we passed to the model contained instructions to generate a company name. For our application, it'd be great if the user only had to provide the description of a company/product, without having to worry about giving the model instructions.

In [16]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template("What is a good name for a company that makes {product}?")
prompt.format(product="colorful socks")

'What is a good name for a company that makes colorful socks?'

# Chains
Now that we've got a model and a prompt template, we'll want to combine the two. Chains give us a way to link (or chain) together multiple primitives, like models, prompts, and other chains.

The simplest and most common type of chain is an LLMChain, which passes an input first to a PromptTemplate and then to an LLM. We can construct an LLM chain from our existing model and prompt template.

In [17]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=prompt)
chain.run("colorful socks")

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')).


'\n\nRainbow Toes.'

# Agents
Our first chain ran a pre-determined sequence of steps. To handle complex workflows, we need to be able to dynamically choose actions based on inputs.

Agents do just this: they use a language model to determine which actions to take and in what order. Agents are given access to tools, and they repeatedly choose a tool, run the tool, and observe the output until they come up with a final answer.

To load an agent, you need to choose a(n):

   1. LLM/Chat model: The language model powering the agent.
   2. Tool(s): A function that performs a specific duty. This can be things like: Google Search, Database lookup, Python REPL, other chains. For a list of predefined tools and their specifications, see the Tools documentation.
   3. Agent name: A string that references a supported agent class. An agent class is largely parameterized by the prompt the language model uses to determine which action to take. Because this notebook focuses on the simplest, highest level API, this only covers using the standard supported agents. 

In [19]:
# You'll need to install the SerpAPI Python package:
!pip install google-search-results

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32077 sha256=e931d37becee508066c37bce071626aba289cf9489246fb225e7f75162a04a94
  Stored in directory: c:\users\irfan\appdata\local\pip\cache\wheels\de\1b\db\6474e3f9e34a03ca54b85d98fc7742001e6fae1ff3881e3ed4
Successfully built google-search-results


In [34]:
%env SERPAPI_API_KEY=525fe40634dc8e328ce2981597c9cd6b579ae2f54fa84bfcc4c48f368b31acdf

env: SERPAPI_API_KEY=525fe40634dc8e328ce2981597c9cd6b579ae2f54fa84bfcc4c48f368b31acdf


In [35]:
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.llms import OpenAI

# The language model we're going to use to control the agent.
llm = OpenAI(openai_api_key="Enter-API-KEY",temperature=0)

# The tools we'll give the Agent access to. Note that the 'llm-math' tool uses an LLM, so we need to pass that in.
tools = load_tools(["serpapi", "llm-math"], llm=llm)

# Finally, let's initialize an agent with the tools, the language model, and the type of agent we want to use.
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

# Let's test it out!
agent.run("What was the high temperature in SF yesterday in Fahrenheit? What is that number raised to the .023 power?")

Error in on_chain_start callback: 'name'


 I need to find the temperature first, then use the calculator to raise it to the .023 power.
Action: Search
Action Input: "High temperature in SF yesterday"
Observation: High: 66.2ºf @1:40 PM Low: 55.04ºf @5:56 AM Approx.
Thought: I now need to use the calculator to raise 66.2 to the .023 power
Action: Calculator
Action Input: 66.2^.023
Observation: Answer: 1.1012343099196273
Thought: I now know the final answer
Final Answer: 1.1012343099196273

> Finished chain.


'1.1012343099196273'

# Memory
The chains and agents we've looked at so far have been stateless, but for many applications it's necessary to reference past interactions. This is clearly the case with a chatbot for example, where you want it to understand new messages in the context of past messages.

The Memory module gives you a way to maintain application state. The base Memory interface is simple: it lets you update state given the latest run inputs and outputs and it lets you modify (or contextualize) the next input using the stored state.

There are a number of built-in memory systems. The simplest of these are is a buffer memory which just prepends the last few inputs/outputs to the current input - we will use this in the example below.

In [36]:
from langchain import OpenAI, ConversationChain

llm = OpenAI(openai_api_key="Enter-API-KEY",temperature=0)
conversation = ConversationChain(llm=llm, verbose=True)

conversation.run("Hi there!")

Error in on_chain_start callback: 'name'


Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi there!
AI:

> Finished chain.


" Hi there! It's nice to meet you. How can I help you today?"

In [37]:
conversation.run("I'm doing well! Just having a conversation with an AI.")

Error in on_chain_start callback: 'name'


Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi there!
AI:  Hi there! It's nice to meet you. How can I help you today?
Human: I'm doing well! Just having a conversation with an AI.
AI:

> Finished chain.


" That's great! It's always nice to have a conversation with someone new. What would you like to talk about?"

In [38]:
conversation.run("Do You know, Who am I?")

Error in on_chain_start callback: 'name'


Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi there!
AI:  Hi there! It's nice to meet you. How can I help you today?
Human: I'm doing well! Just having a conversation with an AI.
AI:  That's great! It's always nice to have a conversation with someone new. What would you like to talk about?
Human: Do You know, Who am I?
AI:

> Finished chain.


" I'm sorry, I don't know who you are. Could you tell me a bit more about yourself?"

In [39]:
conversation.run("I am a lost guy, I am finding my self.")

Error in on_chain_start callback: 'name'


Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi there!
AI:  Hi there! It's nice to meet you. How can I help you today?
Human: I'm doing well! Just having a conversation with an AI.
AI:  That's great! It's always nice to have a conversation with someone new. What would you like to talk about?
Human: Do You know, Who am I?
AI:  I'm sorry, I don't know who you are. Could you tell me a bit more about yourself?
Human: I am a lost guy, I am finding my self.
AI:

> Finished chain.


' I see. It can be difficult to find yourself, but it can also be a rewarding journey. What have you been doing to help you on your journey?'

In [40]:
conversation.run("Suggest me from where I need to start my journey?")

Error in on_chain_start callback: 'name'


Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi there!
AI:  Hi there! It's nice to meet you. How can I help you today?
Human: I'm doing well! Just having a conversation with an AI.
AI:  That's great! It's always nice to have a conversation with someone new. What would you like to talk about?
Human: Do You know, Who am I?
AI:  I'm sorry, I don't know who you are. Could you tell me a bit more about yourself?
Human: I am a lost guy, I am finding my self.
AI:  I see. It can be difficult to find yourself, but it can also be a rewarding journey. What have you been doing to help you on your journey?
Human: Suggest me from where I need to start my journey?
AI:

> Finished chain.


"  That's a great question! It really depends on what you're looking for. What kind of things do you want to explore on your journey?"